<a href="https://colab.research.google.com/github/ALMACihan/IS584_Term_Project/blob/main/Project_Cihan_Alma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **IS 584 Term Project - Cihan Alma**

In [1]:
!pip install -q "transformers==4.52.3" "datasets>=2.0.0" "scikit-learn>=1.3.0" "wandb>=0.16.0"


import wandb
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 52.8 MB/s eta 0:00:00


In [2]:
!pip install -q scikit-learn pandas


In [3]:
import pandas as pd
import csv

df = pd.read_csv(
    "/content/asap_review_dataset.csv",
    on_bad_lines='skip',        # Replaces error_bad_lines
    quoting=csv.QUOTE_ALL,      # Handle quoted fields
    encoding="utf-8",
    engine="python"
)

df.head()


,paper_id,review_text,aspect_label,decision
0,ICLR_2020_1123,This work studies the predictive uncertainty i...,summary,Reject
1,ICLR_2020_1123,This work studies the predictive uncertainty i...,motivation_positive,Reject
2,ICLR_2020_1123,This work studies the predictive uncertainty i...,clarity_positive,Reject
3,ICLR_2020_1123,This work studies the predictive uncertainty i...,soundness_negative,Reject
4,ICLR_2020_1123,This work studies the predictive uncertainty i...,substance_negative,Reject


In [4]:
# Optional: filter to most common aspect labels (optional cleanup)
print(df['aspect_label'].value_counts())

# Drop duplicates
df = df.drop_duplicates(subset=["paper_id", "aspect_label", "review_text"])


aspect_label
summary                           874
clarity_negative                  578
soundness_negative                536
clarity_positive                  466
soundness_positive                435
originality_positive              413
motivation_positive               374
substance_negative                361
originality_negative              310
meaningful_comparison_negative    308
replicability_negative            181
substance_positive                125
motivation_negative               118
meaningful_comparison_positive     46
replicability_positive             16
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split
X_train, X_test, y_train, y_test = train_test_split(
    df["review_text"], df["aspect_label"], test_size=0.2, random_state=42, stratify=df["aspect_label"]
)

# Vectorize
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train
clf = LogisticRegression(max_iter=1000, class_weight="balanced")
clf.fit(X_train_vec, y_train)

# Evaluate
y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))


                                precision    recall  f1-score   support

              clarity_negative       0.05      0.05      0.05        73
              clarity_positive       0.04      0.02      0.03        84
meaningful_comparison_negative       0.01      0.02      0.02        46
meaningful_comparison_positive       0.00      0.00      0.00         9
           motivation_negative       0.02      0.05      0.03        20
           motivation_positive       0.00      0.00      0.00        63
          originality_negative       0.01      0.02      0.02        48
          originality_positive       0.02      0.01      0.02        70
        replicability_negative       0.03      0.07      0.04        30
        replicability_positive       0.00      0.00      0.00         3
            soundness_negative       0.02      0.01      0.02        75
            soundness_positive       0.00      0.00      0.00        70
            substance_negative       0.00      0.00      0.00  

In [6]:
# Sample ~4k examples evenly from all labels
import pandas as pd

df = pd.read_csv(
    "/content/asap_review_dataset.csv",
    on_bad_lines='skip',        # Replaces error_bad_lines
    quoting=csv.QUOTE_ALL,      # Handle quoted fields
    encoding="utf-8",
    engine="python"
)

df_sampled = df.groupby("aspect_label").apply(lambda x: x.sample(n=min(len(x), 300), random_state=42)).reset_index(drop=True)


<ipython-input-6-7e87bcbcdd4a>:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby("aspect_label").apply(lambda x: x.sample(n=min(len(x), 300), random_state=42)).reset_index(drop=True)


In [7]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from datasets import Dataset
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import wandb

wandb.login()


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cihan123 (cihan123-notion) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
# Load the CSV
df = pd.read_csv(
    "/content/asap_review_dataset.csv",
    on_bad_lines='skip',        # Replaces error_bad_lines
    quoting=csv.QUOTE_ALL,      # Handle quoted fields
    encoding="utf-8",
    engine="python"
)
df = df.dropna(subset=["review_text", "aspect_label"])
df = df[df["review_text"].str.strip().astype(bool)]

# Encode labels
le = LabelEncoder()
df["label"] = le.fit_transform(df["aspect_label"])

df = df.groupby("aspect_label").apply(lambda x: x.sample(n=min(len(x), 200), random_state=42)).reset_index(drop=True)


# Create HuggingFace Dataset
dataset = Dataset.from_pandas(df[["review_text", "label"]])
dataset = dataset.train_test_split(test_size=0.2, seed=42)

label2id = {label: i for i, label in enumerate(le.classes_)}
id2label = {i: label for label, i in label2id.items()}
num_labels = len(label2id)


<ipython-input-8-4b1413c6d810>:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("aspect_label").apply(lambda x: x.sample(n=min(len(x), 200), random_state=42)).reset_index(drop=True)


In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example["review_text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2191 [00:00<?, ? examples/s]

Map:   0%|          | 0/548 [00:00<?, ? examples/s]

In [14]:
import transformers
print(transformers.__version__)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"
)



4.52.3


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    report = classification_report(labels, preds, output_dict=True, zero_division=0)
    return {
        "accuracy": report["accuracy"],
        "macro_f1": report["macro avg"]["f1-score"]
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-15-0b56b9585246>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,2.680300,2.670147,0.069343,0.008751
2,2.663500,2.656526,0.072993,0.032842
3,2.620200,2.660017,0.091241,0.060214
4,2.509700,2.669664,0.096715,0.079019
5,2.370000,2.707362,0.091241,0.077398
6,2.221800,2.729981,0.076642,0.063962
7,2.086400,2.763193,0.082117,0.072554
8,1.970200,2.794813,0.083942,0.074201
9,1.886700,2.829512,0.080292,0.071250
10,1.815700,2.839561,0.078467,0.070888


TrainOutput(global_step=2740, training_loss=2.2824438498838107, metrics={'train_runtime': 1236.8269, 'train_samples_per_second': 17.715, 'train_steps_per_second': 2.215, 'total_flos': 2903033577830400.0, 'train_loss': 2.2824438498838107, 'epoch': 10.0})

In [16]:
results = trainer.evaluate()
print("Final Eval Results:", results)


Final Eval Results: {'eval_loss': 2.669663906097412, 'eval_accuracy': 0.09671532846715329, 'eval_macro_f1': 0.07901858646896268, 'eval_runtime': 7.7504, 'eval_samples_per_second': 70.706, 'eval_steps_per_second': 8.903, 'epoch': 10.0}


Second Training
learning_rate=5e-5,
batch_size=16,



In [10]:
import transformers
print(transformers.__version__)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"
)



4.52.3


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    report = classification_report(labels, preds, output_dict=True, zero_division=0)
    return {
        "accuracy": report["accuracy"],
        "macro_f1": report["macro avg"]["f1-score"]
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-11-0b56b9585246>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,2.684800,2.663853,0.071168,0.008859
2,2.672300,2.655320,0.082117,0.032614
3,2.659700,2.664737,0.067518,0.024766
4,2.566900,2.621218,0.096715,0.063156
5,2.400900,2.625093,0.116788,0.077686
6,2.184700,2.654317,0.116788,0.092814
7,1.971000,2.733831,0.133212,0.111047
8,1.788100,2.804752,0.129562,0.114897
9,1.639100,2.862865,0.136861,0.118182
10,1.536800,2.889979,0.129562,0.113530


TrainOutput(global_step=1370, training_loss=2.2104148809057085, metrics={'train_runtime': 261.7705, 'train_samples_per_second': 83.699, 'train_steps_per_second': 5.234, 'total_flos': 2903033577830400.0, 'train_loss': 2.2104148809057085, 'epoch': 10.0})

In [12]:
results = trainer.evaluate()
print("Final Eval Results:", results)


Final Eval Results: {'eval_loss': 2.8628647327423096, 'eval_accuracy': 0.13686131386861314, 'eval_macro_f1': 0.11818237199785897, 'eval_runtime': 2.0179, 'eval_samples_per_second': 271.572, 'eval_steps_per_second': 17.345, 'epoch': 10.0}


In [13]:
import transformers
print(transformers.__version__)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results_baseline",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"
)




4.52.3


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    report = classification_report(labels, preds, output_dict=True, zero_division=0)
    return {
        "accuracy": report["accuracy"],
        "macro_f1": report["macro avg"]["f1-score"]
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-14-0b56b9585246>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,2.678300,2.662677,0.069343,0.008751
2,2.653100,2.635112,0.094891,0.058254
3,2.570500,2.601138,0.114964,0.076317
4,2.412300,2.580222,0.138686,0.113401
5,2.244900,2.611542,0.144161,0.115083
6,2.089000,2.626921,0.136861,0.112029
7,1.960600,2.655709,0.160584,0.129477
8,1.845400,2.692488,0.144161,0.118214
9,1.768100,2.713442,0.138686,0.115852
10,1.705600,2.725155,0.131387,0.111506


TrainOutput(global_step=2740, training_loss=2.1927779705855097, metrics={'train_runtime': 280.0352, 'train_samples_per_second': 78.24, 'train_steps_per_second': 9.784, 'total_flos': 2903033577830400.0, 'train_loss': 2.1927779705855097, 'epoch': 10.0})

In [15]:
import transformers
print(transformers.__version__)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results_batch16",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"
)




4.52.3


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    report = classification_report(labels, preds, output_dict=True, zero_division=0)
    return {
        "accuracy": report["accuracy"],
        "macro_f1": report["macro avg"]["f1-score"]
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-20-0b56b9585246>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,2.691600,2.676575,0.074818,0.018166
2,2.670700,2.636881,0.072993,0.026861
3,2.595900,2.597700,0.091241,0.027918
4,2.499400,2.581933,0.104015,0.081443
5,2.373600,2.619919,0.124088,0.085155
6,2.253400,2.633921,0.118613,0.089175
7,2.124300,2.680526,0.136861,0.108444
8,2.002700,2.696813,0.124088,0.107205
9,1.906500,2.723271,0.124088,0.103709
10,1.843900,2.736184,0.120438,0.103321


TrainOutput(global_step=2740, training_loss=2.29619424638957, metrics={'train_runtime': 279.0009, 'train_samples_per_second': 78.53, 'train_steps_per_second': 9.821, 'total_flos': 2903033577830400.0, 'train_loss': 2.29619424638957, 'epoch': 10.0})

In [21]:
import transformers
print(transformers.__version__)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results_warmup_wd",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.05,           # new
    warmup_steps=500,            # new
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"
)




4.52.3


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    report = classification_report(labels, preds, output_dict=True, zero_division=0)
    return {
        "accuracy": report["accuracy"],
        "macro_f1": report["macro avg"]["f1-score"]
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


<ipython-input-22-0b56b9585246>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,2.688800,2.667192,0.074818,0.015060
2,2.669100,2.641178,0.080292,0.031223
3,2.604300,2.596506,0.098540,0.031102
4,2.488400,2.588819,0.120438,0.088413
5,2.355200,2.626908,0.113139,0.085867
6,2.219600,2.659938,0.122263,0.097772
7,2.072200,2.696597,0.109489,0.089493
8,1.959900,2.724470,0.124088,0.103421
9,1.860600,2.756841,0.113139,0.093718
10,1.793200,2.767034,0.118613,0.101552


TrainOutput(global_step=2740, training_loss=2.271123900030651, metrics={'train_runtime': 284.6389, 'train_samples_per_second': 76.975, 'train_steps_per_second': 9.626, 'total_flos': 2903033577830400.0, 'train_loss': 2.271123900030651, 'epoch': 10.0})